In [2]:
from ngsolve.webgui import Draw

import numpy as np
import ngsolve as ng
import dill
import pickle

motor_npz = np.load('../meshes/motor_pizza_gap.npz', allow_pickle = True)

geoOCC = motor_npz['geoOCC'].tolist()
m = motor_npz['m']; m_new = m
j3 = motor_npz['j3']

geoOCCmesh = geoOCC.GenerateMesh()
mesh = ng.Mesh(geoOCCmesh)
# mesh.Refine()

# Draw(mesh)

In [3]:
# import sys
# sys.path.insert(0,'../../') # adds parent directory

# import pde
# order = 2

# MESH = pde.mesh.netgen(mesh.ngmesh)

# phi_Hcurl = lambda x : pde.hcurl.assemble(MESH, space = 'N0', matrix = 'phi', order = x)
# curlphi_Hcurl = lambda x : pde.hcurl.assemble(MESH, space = 'N0', matrix = 'curlphi', order = x)

# D = lambda x : pde.int.assemble(MESH, order = x)

# myM = lambda x: phi_Hcurl(x)[0] @ D(x) @ phi_Hcurl(x)[0].T + \
#                 phi_Hcurl(x)[1] @ D(x) @ phi_Hcurl(x)[1].T

# myK = lambda x: curlphi_Hcurl(x) @ D(x) @ curlphi_Hcurl(x).T

# R = pde.tools.tree_cotree_gauge(MESH)

In [4]:
HCurl = ng.HCurl(mesh, order = 2, nograds = True, dirichlet = 'left|right|stator_outer')
# HCurl = ng.HCurl(mesh, order = 2, nograds = True)

u,v = HCurl.TnT()

K = ng.BilinearForm(HCurl)
# K += (ng.curl(u)*ng.curl(v)+1e-5*u*v)*ng.dx
K += (ng.curl(u)*ng.curl(v)+0*u*v)*ng.dx
K.Assemble()

f = ng.LinearForm(HCurl)
f += ng.InnerProduct(ng.CF(ng.x**2),ng.curl(v))*ng.dx
# f += ng.CF((1,1))*v*ng.dx
f.Assemble()

# rows,cols,vals = K.mat.COO()
# import scipy.sparse as sp
# ngK = sp.csr_matrix((vals,(rows,cols)))

# np.linalg.matrix_rank(ngK.A)
# print(ngK.shape)

In [5]:
from mst5 import *
newFreeDofs = CoTreeBitArray(mesh, HCurl)

In [6]:
A = ng.GridFunction(HCurl)
A.vec.data = K.mat.Inverse(freedofs = newFreeDofs, inverse = 'sparsecholesky')*f.vec
# A.vec.data = K.mat.Inverse(HCurl.FreeDofs(), inverse = 'sparsecholesky')*f.vec

Draw(A, vectors={"grid_size":100})
Draw(ng.curl(A), mesh)
A.vec.data.FV().NumPy().max()
# np.where(FreeDofsCotreeGauge)[0].shape

ng.Integrate(ng.curl(A),mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

2.8933523386390172e-05